<a href="https://colab.research.google.com/github/bonly/AI/blob/main/torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#安装

如果没有nvidia显示卡的：
pip3 install torch torchvision #torchaudio

In [ ]:
!pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu129

#核心概念

## 张量（Tensor） ：类似于 NumPy 的多维数组，不同的是 PyTorch 的张量可以利用 GPU 加速计算。

In [ ]:
import torch
x = torch.tensor([1, 2, 3])  # 创建一个一维张量
y = torch.tensor([[4, 5], [6, 7]])  # 创建一个二维张量

## 自动求导(Autograd):
>PyTorch的自动微分引擎，能够记录张量操作，并自动计算梯度。
### .requires_grad_(True):
>设置此属性为True，表示需要跟踪该张量的操作，以便后续自动求导。
### .backward():
>在计算图的某个张量上调用 **.backward()，PyTorch会自动沿着计算历史，计算所有依赖张量的梯度，并将结果存储在每个张量的.grad** 属性中。
### 梯度(Gradient):
>函数的导数，表示函数值变化对输入变量变化的敏感程度。在深度学习中，梯度用于指导模型参数的更新方向，使模型朝着最小化损失函数的方向优化。


> 自动求导（Autograd） ：PyTorch 的自动求导功能可以自动计算张量的梯度，这对于神经网络的训练至关重要。在张量上调用 .backward() 方法可以计算梯度；使用 torch.no_grad() 可以停止梯度追踪。

In [ ]:
x = torch.ones(2, 2, requires_grad=True) #创建一个2x2的全1张量，并梯度跟踪
y = x * 3
y.backward(torch.ones_like(x)) #创建一个与输入张量（tensor）形状相同、但所有元素值为1的新张量
print(x.grad)

tensor([[3., 3.],
        [3., 3.]])


In [ ]:
import torch

# 创建一个需要求导的张量
x = torch.tensor(2.0, requires_grad=True)
# 进行一系列计算
y = x + 2
z = y * y * 3
# 目标函数
target = torch.tensor(30.0)
# 计算损失
loss = (z - target).pow(2)

# 自动求导
loss.backward()

# 打印梯度
print(x.grad) # 输出: tensor(48.)

tensor(864.)


>x是输入，y, z, loss是中间变量和最终的损失。通过调用loss.backward()，PyTorch自动计算了x的梯度，并存储在x.grad中

#构建神经网络

##定义网络结构 ：使用 torch.nn.Module 来定义神经网络的结构

In [ ]:
import torch
import torch.nn as nn
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 10)  # 输入层到隐藏层的连接，输入维度为4，输出维度为10
        self.fc2 = nn.Linear(10, 4)  # 隐藏层到输出层的连接，输入维度为10，输出维度为4
        self.relu = nn.ReLU()  # 激活函数 ReLU：如果输入大于 0，输出等于输入；否则输出 0。
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
net = Net() #创建一个 Net 类的实例（即一个具体的神经网络对象）

### self.fc1 = nn.Linear(4, 10)

* 作用：定义第一个全连接层（也叫线性层）。
* 简单解释：

    * 输入维度是 4，输出维度是 10。
    * 例子：如果输入是 [a, b, c, d]（4个数），这层会输出 10 个数。

### self.fc2 = nn.Linear(10, 4)

* 作用：定义第二个全连接层。
* 简单解释：

    * 输入维度是 10（接收上一层的输出），输出维度是 4。
    * 例子：将 10 个数转换为 4 个数。

### self.relu = nn.ReLU()

* 作用：定义 ReLU 激活函数。
* 简单解释：

    * ReLU：如果输入大于 0，输出等于输入；否则输出 0。
    * 例子：ReLU(2) = 2，ReLU(-1) = 0。

### forward 函数
* 作用：定义数据如何通过网络（即“前向传播”）。
* 简单解释：

    * 输入 x 先经过 fc1 层，再经过 ReLU 激活，最后经过 fc2 层。

### 整体流程

* 输入 4 个数 → fc1 层 → 得到 10 个数

* 10 个数 → ReLU 激活 → 非负的 10 个数

* 10 个数 → fc2 层 → 得到 4 个数（最终输出）

## 定义损失函数和优化器 ：
选择合适的损失函数和优化器对于模型的训练效果至关重要。

常见的损失函数有均方误差损失（MSELoss）、交叉熵损失（CrossEntropyLoss）等；

常用的优化器有随机梯度下降（SGD）、Adam 等。

In [ ]:
criterion = nn.MSELoss()  # 定义均方误差损失函数
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)  # 定义随机梯度下降优化器，学习率为0.01

### 损失函数（Loss Function）
代码：

criterion = nn.MSELoss()

含义：

* 损失函数用来衡量模型的预测值和真实值之间的差距。

* MSELoss 是“均方误差损失”的简称，计算的是预测值和真实值之间的平方差的平均值。

* 举例：
假设你预测一个房子的价格是 300 万元，实际价格是 280 万元，MSELoss 会计算 (300 - 280)^2 = 400，然后对所有样本的平方差取平均值，作为模型的“错误程度”。

为什么用均方误差？

* 平方操作会放大较大的错误，让模型更关注大的误差。

### 优化器（Optimizer）

代码：

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

含义：

* 优化器用来根据损失函数的结果，调整模型的参数（权重和偏置），使模型更准确。

* SGD 是“随机梯度下降”的简称，是最常用的优化方法之一。

   * net.parameters()：模型中需要调整的所有参数。
   
   * lr=0.01：学习率（learning rate），决定每次调整参数的步子大小。

      * 学习率太大：可能调整过头，模型不稳定。

      * 学习率太小：调整太慢，训练时间长。

举例：

- 假设模型预测错了，损失函数告诉你“错了 20 万元”，SGD 会根据学习率（0.01）调整参数，比如把权重减少 0.01 * 20 = 0.2 单位，让下次预测更接近真实值。

### 总结

* 损失函数告诉模型“错了多少”。

* 优化器告诉模型“如何改进”。

这两者配合使用，让模型在训练过程中不断提高准确度。

如果你想更直观地理解，可以想象成：

* 损失函数是“考试成绩单”，告诉你哪里错了。

* 优化器是“学习方法”，告诉你如何改进成绩。

## 训练网络 ：使用训练数据对网络进行训练，通过前向传播计算输出，计算损失函数的值，然后进行反向传播更新网络的参数。

In [ ]:
for epoch in range(1000):  # 训练1000个周期
    optimizer.zero_grad()  # 清空梯度
    output = net(x_data)  # 前向传播
    loss = criterion(output, y_data)  # 计算损失
    loss.backward()  # 反向传播
    optimizer.step()  # 更新参数
    if epoch % 100 == 99:  # 每100个周期打印一次损失
        print('Epoch: {}, Loss: {}'.format(epoch+1, loss.item()))

### optimizer.zero_grad()

* 含义：每次训练前，清空之前的梯度（误差反馈）

### 损失函数（loss = criterion(output, y_data)）

* 含义：计算模型预测结果（output）和真实答案（y_data）之间的差距。

### 反向传播（loss.backward()）

* 含义：根据损失，计算每个参数（权重）需要调整的方向和大小。

* 例子：告诉模型“猜错是因为哪些地方没学好”，并标记出来。

### 更新参数（optimizer.step()）

* 含义：根据反向传播的结果，实际调整模型的参数（权重）。

* 例子：根据标记，调整模型的“学习笔记”，让下次猜得更准。